# Summary and Important Functions

This jupyter notebook contains basic implementation of how UniswapV3 changes with respect to a liquidity position while ignoring fees. Essentially an LP position is defined by an interval $[a,b]$ where $a < b$ where $a,b$ represent the price of an asset $A$. The composition of the LP positions is given by an Asset A and an Asset B where the LP position is completely $B$ if the current price $c > b$ and is completely $A$ if $c < a$. The composition of the LP position is defined by the equation $a*b=k$ with respect to the correct translation. 


The main function can be found in the function $make\_table$. This creates a table the gives an range of possible values for the LP position. 

To see an example run all the cells in this notebook and place the following code in a python cell. Note that the prices given are in relative values. I.E. The following example is given for the WETH/USDC pair where WETH has 18 decimal places and USDC has 6 decimal places and the price of USDC in WETH must be converted correctly. 

#### lower bound price
a = 1000.7
#### upper bound price
b = 1270.4
#### current price
c = 1210.54
        
#### input values are USDC units divided by WETH units
ticklower = price_to_tick(1/a*10**12)
tickupper = price_to_tick(1/b*10**12)
currenTick = price_to_tick(1/c*10**12)

#### USDC decimal places
decimals0 = 6 
#### WETH decimal places
decimals1 = 18

#### Amount of USDC
amount0 = 8250
#### Amount of WETH
amount1 = 1.833
    


make_table(15,price_to_tick(1/1500*10**12),currenTick,ticklower,tickupper,amount0,amount1,decimals0,decimals1).show()

In [8]:
import math
from datascience import *


#Converts price x/y to tick
def price_to_tick(p):
    base = math.sqrt(1.0001)
    return math.floor(math.log(p, 1.0001))

# Converts the sqrt price x/y to tick
def sqrt_price_to_tick(sqrtprice):
    return math.log((sqrtprice/(2**96))**2,1.0001)
 
#Converts tick to sqrt price x/y
def tick_to_sqrtprice(tick):
    return 1.0001**(tick/2)*(2**96)

# Converts price to sqrt price x/y
q96 = 2**96
def price_to_sqrtp(p):
    return int(math.sqrt(p)*q96)

                               
def sqrtp_to_price(sqrtp):
    return (sqrtp/q96)**2



#Example: Price to Tick and Tick to Price
#print((1/1287.57)*10**12)
#print(price_to_tick((1/1287.57)*10**12))
#print(1/sqrtp_to_price(tick_to_sqrtprice(204715))*10**12)



#Example Price to Square root Price and Square root price to price
#print((1/1287.57)*10**12)
#price_to_sqrtp((1/1287.57)*10**12)
#sqrtp_to_price(2207975037607419429762796598853632)

In [9]:
import math 


'''get_liquidity function'''
#Use 'get_liquidity' function to calculate liquidity as a function of amounts and price range
def get_liquidity0(sqrtA,sqrtB,amount0,decimals):
    
    if (sqrtA > sqrtB):
          (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity = int(amount0*((2**96*(sqrtB*sqrtA)/(sqrtB-sqrtA))/10**decimals))
    return liquidity

def get_liquidity1(sqrtA,sqrtB,amount1,decimals):
    
    if (sqrtA > sqrtB):
        (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity = int(amount1/((sqrtB-sqrtA)/2**96/10**decimals))
    return liquidity


# Gets Liquidity L from UniswapV3 white paper. 
def get_liquidity(tick,tickA,tickB,amount0,amount1,decimal0,decimal1):
    
        sqrt  = int(1.0001**(tick/2)*(2**96))
        sqrtA = int(1.0001**(tickA/2)*(2**96))
        sqrtB = int(1.0001**(tickB/2)*(2**96))
        #print(sqrt,sqrtA,sqrtB)
        
        if (sqrtA > sqrtB):
            (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
        if sqrt<=sqrtA:
            liquidity0=get_liquidity0(sqrtA,sqrtB,amount0,decimal0)   
            return liquidity0        
        elif sqrt<sqrtB and sqrt>sqrtA:
            liquidity0 = get_liquidity0(sqrt,sqrtB,amount0,decimal0)
            
            liquidity1 = get_liquidity1(sqrtA,sqrt,amount1,decimal1)
            
            liquidity  = liquidity0 if liquidity0<liquidity1 else liquidity1
         
            return liquidity
        else:
            liquidity1 = get_liquidity1(sqrtA,sqrtB,amount1,decimal1)
            return liquidity1



        
# Example 
#
# Get Liquidity for Example Values

#a = 1226.7
#b = 1358.4
#c = 1310.54
        
#ticklower = price_to_tick(1/a*10**12)
#tickupper = price_to_tick(1/b*10**12)
#currenTick = price_to_tick(1/c*10**12)

#decimals0 = 6 
#decimals1 = 18

#amount0 = 6510 
#amount1 = 2.712

#get_liquidity(currenTick,ticklower,tickupper,amount0,amount1,decimals0,decimals1)


In [10]:

#sqrtP: format X96 = int(1.0001**(tick/2)*(2**96))
#liquidity: int
#sqrtA = price for lower tick
#sqrtB = price for upper tick
'''get_amounts function'''
#Use 'get_amounts' function to calculate amounts as a function of liquitidy and price range
def get_amount0(sqrtA,sqrtB,liquidity,decimals):
    
    if (sqrtA > sqrtB):
          (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    amount0=((liquidity*2**96*(sqrtB-sqrtA)/sqrtB/sqrtA)/10**decimals)
    
    return amount0

def get_amount1(sqrtA,sqrtB,liquidity,decimals):
    
    if (sqrtA > sqrtB):
        (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    amount1=liquidity*(sqrtB-sqrtA)/2**96/10**decimals
    
    return amount1

# Get amount of Token A and Token B given the input values
def get_amounts(tick,tickA,tickB,liquidity,decimal0,decimal1):

    sqrt  = int(1.0001**(tick/2)*(2**96))
    sqrtA = int(1.0001**(tickA/2)*(2**96))
    sqrtB = int(1.0001**(tickB/2)*(2**96))

    if (sqrtA > sqrtB):
        (sqrtA,sqrtB)=(sqrtB,sqrtA)

    if sqrt<=sqrtA:

        amount0 = get_amount0(sqrtA,sqrtB,liquidity,decimal0)
        return [amount0,0]
   
    elif sqrt<sqrtB and sqrt>sqrtA:
        amount0 = get_amount0(sqrt,sqrtB,liquidity,decimal0)
        amount1 = get_amount1(sqrtA,sqrt,liquidity,decimal1)
        return [amount0,amount1]
    
    else:
        amount1=get_amount1(sqrtA,sqrtB,liquidity,decimal1)
        return [0,amount1]

    
    
# Example 
#
# Get Liquidity for Example Values

'''
a = 1000.7
b = 1270.4
c = 1210.54
        
ticklower = price_to_tick(1/a*10**12)
tickupper = price_to_tick(1/b*10**12)
currenTick = price_to_tick(1/c*10**12)

decimals0 = 6 
decimals1 = 18

amount0 = 8250 
amount1 = 1.833
'''
    
    

# Tick Current Price, Tick lower, Tick Upper, Amount 0, Amount 1, decimal 0, decimal 1
#l = get_liquidity(currenTick,ticklower,tickupper,amount0,amount1,decimals0,decimals1)
#get_amounts(currenTick,ticklower,tickupper,l,decimals0,decimals1)

'\na = 1000.7\nb = 1270.4\nc = 1210.54\n        \nticklower = price_to_tick(1/a*10**12)\ntickupper = price_to_tick(1/b*10**12)\ncurrenTick = price_to_tick(1/c*10**12)\n\ndecimals0 = 6 \ndecimals1 = 18\n\namount0 = 8250 \namount1 = 1.833\n'

In [13]:
# Give the parameters of a UniswapV3 position with bounds [a,b] and initial price c
# this function will find the updated amounts at price corresponding to tick_delta
#

def get_amounts_delta(tick_delta,tick,tickA,tickB,amount0,amount1,decimal0,decimal1):
    liq = get_liquidity(tick,tickA,tickB,amount0,amount1,decimal0,decimal1)
    #old = get_amounts(tick,tickA,tickB,liq,decimal0,decimal1)
    delta = get_amounts(tick_delta,tickA,tickB,liq,decimal0,decimal1)
    #print(old)
    #print(New)
    return delta


# Example of price 

a = 1000.7
b = 1270.4
c = 1210.54
        
ticklower = price_to_tick(1/a*10**12)
tickupper = price_to_tick(1/b*10**12)
currenTick = price_to_tick(1/c*10**12)

decimals0 = 6 
decimals1 = 18

amount0 = 8250 
amount1 = 1.833

get_amounts_delta(price_to_tick(1/1240*10**12),currenTick,ticklower,tickupper,amount0,amount1,decimals0,decimals1)


[9572.371028769514, 0.9128644682612767]

In [14]:

def make_table(amounts,tick_delta,tick,tickA,tickB,amount0,amount1,decimal0,decimal1):
    IP = Table(['USDC Amount','Eth Amount','Eth Price','LP Position Total','Dollars if in Eth','Divergence','LP Eth Amount'])
    
    first = get_amounts_delta(tick,tick,tickA,tickB,amount0,amount1,decimal0,decimal1)
    initialPrice = 1/sqrtp_to_price(tick_to_sqrtprice(tick)) *10 ** (decimal1-decimal0)
    total = first[0] + first[1]*initialPrice
    amount_in_eth = total / initialPrice
    div = amount_in_eth * initialPrice - total
    row = first + [initialPrice] + [total] + [amount_in_eth] + [div]+[total/initialPrice]
    #print(row)
    IP=IP.with_row(row)
    #print(list(first)+[1366])
    ok = (b-a)/amounts
    current = 1/sqrtp_to_price(tick_to_sqrtprice(tickA)) *10 ** (decimal1-decimal0)
    while current < b:
        #    figet_amounts_delta(tick_delta,tick,tickA,tickB,amount0,amount1,decimal0,decimal1)
        first=get_amounts_delta(price_to_tick(1/current*10**12),currenTick,ticklower,tickupper,amount0,amount1,decimal0,decimal1)
        total = first[0] + first[1]*current
        currentEth= current * amount_in_eth
        div =  -currentEth + total
        row = first + [current] + [total] + [currentEth] + [div]+[total/current]
        #print(row)
        IP=IP.with_row(row)
        current = current + ok

    return IP
    
make_table(15,price_to_tick(1/1500*10**12),currenTick,ticklower,tickupper,amount0,amount1,decimals0,decimals1).show()

USDC Amount,Eth Amount,Eth Price,LP Position Total,Dollars if in Eth,Divergence,LP Eth Amount
8444.98,1.833,1210.57,10664,8.80903,0,8.80903
0,9.50575,1000.7,9512.42,8815.21,697.21,9.50575
760.141,8.75291,1018.68,9676.57,8973.6,702.974,9.49911
1509.9,8.02338,1036.66,9827.43,9131.99,695.447,9.47988
2253.23,7.31254,1054.64,9965.35,9290.37,674.974,9.44903
2989.84,6.62004,1072.62,10090.6,9448.76,641.885,9.40745
3719.47,5.9455,1090.6,10203.6,9607.14,596.496,9.35597
4441.82,5.28858,1108.58,10304.6,9765.53,539.11,9.29533
5161.11,4.64495,1126.56,10393.9,9923.92,470.017,9.22624
5877.18,4.01438,1144.54,10471.8,10082.3,389.497,9.14934


In [5]:
'''

#print(ticklower,tickupper,currenTick)
#print(tickupper-ticklower,tickupper-tickupper,(tickupper-currenTick)/(tickupper-ticklower))


def ETHUSDCLP(a,b,c,amountUSDC):
    
    decimals0 = 6 
    decimals1 = 18
    
    ticklower = price_to_tick(1/a*10**12)
    tickupper = price_to_tick(1/b*10**12)
    currenTick = price_to_tick(1/c*10**12)
    usdcAmountpercentage = (tickupper-currenTick)/(tickupper-ticklower)
    amount0 = amountUSDC * (1-usdcAmountpercentage)
    amount1 = (amountUSDC *(usdcAmountpercentage)) / c
    
    #print(amount0,amount1,c)
    l = get_liquidity(currenTick,ticklower,tickupper,amount0,amount1,decimals0,decimals1)
    amounts= get_amounts(currenTick,ticklower,tickupper,l,decimals0,decimals1)
    #print(amounts)
    
    return amounts

    
ETHUSDCLP(a,b,c,2869.39)'''

'\n\n#print(ticklower,tickupper,currenTick)\n#print(tickupper-ticklower,tickupper-tickupper,(tickupper-currenTick)/(tickupper-ticklower))\n\n\ndef ETHUSDCLP(a,b,c,amountUSDC):\n    \n    decimals0 = 6 \n    decimals1 = 18\n    \n    ticklower = price_to_tick(1/a*10**12)\n    tickupper = price_to_tick(1/b*10**12)\n    currenTick = price_to_tick(1/c*10**12)\n    usdcAmountpercentage = (tickupper-currenTick)/(tickupper-ticklower)\n    amount0 = amountUSDC * (1-usdcAmountpercentage)\n    amount1 = (amountUSDC *(usdcAmountpercentage)) / c\n    \n    #print(amount0,amount1,c)\n    l = get_liquidity(currenTick,ticklower,tickupper,amount0,amount1,decimals0,decimals1)\n    amounts= get_amounts(currenTick,ticklower,tickupper,l,decimals0,decimals1)\n    #print(amounts)\n    \n    return amounts\n\n    \nETHUSDCLP(a,b,c,2869.39)'

In [6]:
'''
a = 1000
b = 1270
c = 1193
        
ticklower = price_to_tick(1/a*10**12)
tickupper = price_to_tick(1/b*10**12)
currenTick = price_to_tick(1/c*10**12)

decimals0 = 6 
decimals1 = 18

amount0 = 7582
amount1 = 2.38

make_table(20,price_to_tick(1/c*10**12),currenTick,ticklower,tickupper,amount0,amount1,decimals0,decimals1).show()'''

'\na = 1000\nb = 1270\nc = 1193\n        \nticklower = price_to_tick(1/a*10**12)\ntickupper = price_to_tick(1/b*10**12)\ncurrenTick = price_to_tick(1/c*10**12)\n\ndecimals0 = 6 \ndecimals1 = 18\n\namount0 = 7582\namount1 = 2.38\n\nmake_table(20,price_to_tick(1/c*10**12),currenTick,ticklower,tickupper,amount0,amount1,decimals0,decimals1).show()'

In [ ]:
'''
#### lower bound price
a = 1000.7
#### upper bound price
b = 1270.4
#### current price
c = 1210.54
        
#### input values are USDC units divided by WETH units
ticklower = price_to_tick(1/a*10**12)
tickupper = price_to_tick(1/b*10**12)
currenTick = price_to_tick(1/c*10**12)

#### USDC decimal places
decimals0 = 6 
#### WETH decimal places
decimals1 = 18

#### Amount of USDC
amount0 = 8250
#### Amount of WETH
amount1 = 1.833
    


make_table(15,price_to_tick(1/1500*10**12),currenTick,ticklower,tickupper,amount0,amount1,decimals0,decimals1).show()
'''